In [1]:
# Import necessary libraries
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

# Function to download and load the Titanic dataset
def load_titanic_data():
    tarball_path = Path("datasets/titanic.tgz")

    # Check if the dataset archive exists locally, if not, download and extract it
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/titanic.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as titanic_tarball:
            titanic_tarball.extractall(path="datasets")

    # Load and return the train and test data as pandas DataFrames
    return [pd.read_csv(Path("datasets/titanic") / filename)
            for filename in ("train.csv", "test.csv")]

In [2]:
# Load the dataset
train_data, test_data = load_titanic_data()

In [3]:
# Show the first few rows of the training data (for initial inspection)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Set 'PassengerId' as the index for easier tracking and referencing
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")

In [5]:
# Display structure and missing values in training data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [6]:
# Check the median age of female passengers (may help in imputation strategy)
train_data[train_data["Sex"]=="female"]["Age"].median()

27.0

In [7]:
# Generate descriptive statistics of numerical features
train_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699113,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526507,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.416700,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
# Check how many passengers survived (target variable distribution)
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [9]:
# Explore passenger class distribution
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [10]:
# Explore gender distribution
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [11]:
# Check how many people boarded from each port
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [12]:
# Create a pipeline for numerical features
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Numerical pipeline: impute missing values with median, then scale the features
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [13]:
# Create a pipeline for categorical features
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [15]:
# Categorical pipeline: encode ordinal features, impute with most frequent, then one-hot encode
cat_pipeline = Pipeline([
    ("ordinal_encoder", OrdinalEncoder()),
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("cat_encoder", OneHotEncoder(sparse=False)),
])

In [16]:
# Combine numerical and categorical pipelines
from sklearn.compose import ColumnTransformer

# Specify which columns are numerical and which are categorical
num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

# Combine both pipelines using ColumnTransformer
preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [65]:
# Prepare the data for training
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# Apply preprocessing to training data
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

array([[-0.56573582,  0.43279337, -0.47367361, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.6638609 ,  0.43279337, -0.47367361, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25833664, -0.4745452 , -0.47367361, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.10463705,  0.43279337,  2.00893337, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25833664, -0.4745452 , -0.47367361, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.20276213, -0.4745452 , -0.47367361, ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
# Extract target variable
y_train = train_data["Survived"]

In [19]:
# Train a Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Create and train the classifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [20]:
# Prepare test data and make predictions
# Apply the same preprocessing to the test data (important: use transform, not fit_transform)
X_test = preprocess_pipeline.transform(test_data)

# Predict survival on the test set
y_pred = forest_clf.predict(X_test)

In [21]:
# Evaluate the model using cross-validation
from sklearn.model_selection import cross_val_score

# Evaluate model accuracy with 10-fold cross-validation
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)

# Print the average accuracy
forest_scores.mean()

0.8137578027465668

In [32]:
# Create a DataFrame with PassengerId and corresponding predictions
submission_df = pd.DataFrame({
    "PassengerId": test_data.index,  # Use index since we set it earlier
    "Survived": y_pred               # Predictions from your model
})

# Save to a CSV file (don't include the index column in the CSV)
submission_df.to_csv("titanic_submission.csv", index=False)

In [22]:
# Improve the above random forest model with feature engineering
# Feature: Age bucket (e.g. 0, 15, 30...)
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
test_data["AgeBucket"] = test_data["Age"] // 15 * 15

In [23]:
# Feature: Relatives Onboard
train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
test_data["RelativesOnboard"] = test_data["SibSp"] + test_data["Parch"]

In [24]:
# Prepare features for the pipeline
num_attribs = ["Age", "SibSp", "Parch", "Fare", "RelativesOnboard"]
cat_attribs = ["Pclass", "Sex", "Embarked", "AgeBucket"]

In [25]:
# Create preprocessing pipelines
# Pipeline for numerical features
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [27]:
# Pipeline for categorical features
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(sparse=False, handle_unknown="ignore"))
])

In [28]:
# Combine numerical and categorical transformers
preprocess_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])

In [29]:
# Preprocess the data
X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data["Survived"]
X_test = preprocess_pipeline.transform(test_data)

In [30]:
# Train the Random Forest model
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [31]:
# Evaluate using 10-fold cross-validation
scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
print("Average cross-validation accuracy:", scores.mean())

Average cross-validation accuracy: 0.8047565543071162


In [33]:
# Predict on the test set
y_pred = forest_clf.predict(X_test)

In [34]:
# Export to CSV for Kaggle submission
output = pd.DataFrame({
    "PassengerId": test_data.index,
    "Survived": y_pred
})
output.to_csv("titanic_submission02.csv", index=False)

In [35]:
# Improve the Random Forest model with grid search
# Grid Search for Best Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [36]:
# Define parameter grid to search
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt", "log2"]
}

In [37]:
# Create base model
base_model = RandomForestClassifier(random_state=42)

In [38]:
# Grid search with cross-validation
grid_search = GridSearchCV(estimator=base_model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

In [39]:
# Train grid search
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=2)

In [40]:
# Output best params and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Score: 0.8294143493817085


In [41]:
# Predict using best model and export CSV
# Get best estimator
best_model = grid_search.best_estimator_

# Make predictions on test set
y_pred = best_model.predict(X_test)

In [42]:
# Create submission DataFrame
submission = pd.DataFrame({
    "PassengerId": test_data.index,
    "Survived": y_pred
})

# Export to CSV for Kaggle submission
submission.to_csv("titanic_gridsearch_submission.csv", index=False)

In [66]:
# Logistic Regression Model
# Feature Engineering
# Create new features from existing ones to boost predictive power
train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
train_data["IsAlone"] = (train_data["RelativesOnboard"] == 0).astype(int)
train_data["Title"] = train_data["Name"].str.extract(r',\s*([^\.]+)\.')
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
train_data["Deck"] = train_data["Cabin"].str[0].fillna("U")

test_data["RelativesOnboard"] = test_data["SibSp"] + test_data["Parch"]
test_data["IsAlone"] = (test_data["RelativesOnboard"] == 0).astype(int)
test_data["Title"] = test_data["Name"].str.extract(r',\s*([^\.]+)\.')
test_data["AgeBucket"] = test_data["Age"] // 15 * 15
test_data["Deck"] = test_data["Cabin"].str[0].fillna("U")

In [67]:
# Define numerical and categorical attributes for processing
num_attribs = ["Age", "SibSp", "Parch", "Fare", "RelativesOnboard"]
cat_attribs = ["Pclass", "Sex", "Embarked", "Title", "Deck", "IsAlone"]

# Pipeline for numerical attributes
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline for categorical attributes
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(sparse=False, handle_unknown="ignore"))
])

# Combine both pipelines into a single preprocessing step
preprocess_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])

In [68]:
# Prepare the data for training
X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data["Survived"]
X_test = preprocess_pipeline.transform(test_data)

In [69]:
# Train Logistic Regression Model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Train and evaluate a logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [70]:
# Evaluate with cross-validation
scores = cross_val_score(log_reg, X_train, y_train, cv=5, scoring='accuracy')
print(f"Logistic Regression Accuracy: {scores.mean():.4f}")

Logistic Regression Accuracy: 0.8260


In [49]:
# Predict and Export CSV
# Predict on the test dataset and save submission file
y_pred = log_reg.predict(X_test)
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": y_pred})
output.to_csv("logistic_regression_submission.csv", index=False)
print("Submission saved as logistic_regression_submission.csv")

Submission saved as logistic_regression_submission.csv


In [50]:
# Gradient Boosting Classifier
# Train Gradient Boosting Model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [51]:
# Evaluate with cross-validation
scores = cross_val_score(gb_clf, X_train, y_train, cv=5, scoring='accuracy')
print(f"Gradient Boosting Accuracy: {scores.mean():.4f}")

Gradient Boosting Accuracy: 0.8272


In [52]:
# Predict and Export CSV
y_pred = gb_clf.predict(X_test)
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": y_pred})
output.to_csv("gradient_boosting_submission.csv", index=False)
print("Submission saved as gradient_boosting_submission.csv")

Submission saved as gradient_boosting_submission.csv


In [53]:
# Stacking Classifier
# Feature Engineering
def age_group(age):
    if pd.isnull(age):
        return "Unknown"
    elif age < 13:
        return "Child"
    elif age < 18:
        return "Teen"
    elif age < 35:
        return "YoungAdult"
    elif age < 60:
        return "Adult"
    else:
        return "Senior"

for data in (train_data, test_data):
    data["RelativesOnboard"] = data["SibSp"] + data["Parch"]
    data["IsAlone"] = (data["RelativesOnboard"] == 0).astype(int)
    data["Title"] = data["Name"].str.extract(r',\s*([^\.]+)\.')
    data["Deck"] = data["Cabin"].str[0].fillna("U")
    data["AgeGroup"] = data["Age"].apply(age_group)
    data["IsMarried"] = (data["Title"] == "Mrs").astype(int)
    data["GroupSize"] = data.groupby("Ticket")["Name"].transform("count")

In [55]:
# Preprocessing Pipelines
num_attribs = ["Age", "SibSp", "Parch", "Fare", "RelativesOnboard", "GroupSize"]
cat_attribs = ["Pclass", "Sex", "Embarked", "Title", "Deck", "IsAlone", "AgeGroup", "IsMarried"]

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse=False))
])

preprocess_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])

X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data["Survived"]
X_test = preprocess_pipeline.transform(test_data)

In [56]:
# Stacking Classifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

stack_model = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('svc', SVC(probability=True, random_state=42)),
        ('gb', GradientBoostingClassifier(random_state=42))
    ],
    final_estimator=LogisticRegression()
)
stack_model.fit(X_train, y_train)

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(random_state=42))],
                   final_estimator=LogisticRegression())

In [57]:
# Cross-validation Evaluation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(stack_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Stacked Model Accuracy: {scores.mean():.4f}")

Stacked Model Accuracy: 0.8305


In [58]:
# Predict and Export CSV
y_pred = stack_model.predict(X_test)
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": y_pred})
output.to_csv("stacking_submission.csv", index=False)
print("Submission saved as stacking_submission.csv")

Submission saved as stacking_submission.csv


In [59]:
# SVC (Support Vector Classifier)
# Feature Engineering
train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
train_data["IsAlone"] = (train_data["RelativesOnboard"] == 0).astype(int)
train_data["Title"] = train_data["Name"].str.extract(r',\s*([^\.]+)\.')
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
train_data["Deck"] = train_data["Cabin"].str[0].fillna("U")

test_data["RelativesOnboard"] = test_data["SibSp"] + test_data["Parch"]
test_data["IsAlone"] = (test_data["RelativesOnboard"] == 0).astype(int)
test_data["Title"] = test_data["Name"].str.extract(r',\s*([^\.]+)\.')
test_data["AgeBucket"] = test_data["Age"] // 15 * 15
test_data["Deck"] = test_data["Cabin"].str[0].fillna("U")

In [61]:
# Preprocessing Pipelines

num_attribs = ["Age", "SibSp", "Parch", "Fare", "RelativesOnboard"]
cat_attribs = ["Pclass", "Sex", "Embarked", "Title", "Deck", "IsAlone", "AgeBucket"]

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse=False))
])

preprocess_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])

X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data["Survived"]
X_test = preprocess_pipeline.transform(test_data)

In [62]:
# Train Support Vector Classifier Model
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc_clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svc_clf.fit(X_train, y_train)

SVC(random_state=42)

In [63]:
# Evaluate with cross-validation
scores = cross_val_score(svc_clf, X_train, y_train, cv=5, scoring='accuracy')
print(f"Support Vector Classifier Accuracy: {scores.mean():.4f}")

Support Vector Classifier Accuracy: 0.8339


In [64]:
# Predict and Export CSV
y_pred = svc_clf.predict(X_test)
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": y_pred})
output.to_csv("svc_submission.csv", index=False)
print("Submission saved as svc_submission.csv")

Submission saved as svc_submission.csv
